<a href="https://colab.research.google.com/github/Yukselendincer/datasceinceproject/blob/main/Camlux_Fabrikas%C4%B1_Dijital_%C4%B0kiz_D%C3%B6n%C3%BC%C5%9F%C3%BCm%C3%BC_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükle
!pip install dash plotly flask-ngrok requests pandas numpy -q

import random
import time
import threading
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import json

In [ ]:
class ColabCamFabrikasi:
    def __init__(self):
        self.uretim_verisi = []
        self.ik_verisi = []
        self.sensor_verisi = {}
        self.calisanlar = [f"EMP{100+i}" for i in range(30)]
        self.bolumler = ["Ergitme", "Sekillendirme", "Kalite_Kontrol", "Ambalaj"]
        self._veri_gecmisi = []

    def gercek_zamanli_sensor_verisi(self):
        """Colab için optimize edilmiş sensör verisi"""
        # Trend ekleyerek daha gerçekçi veri üret
        trend = np.sin(time.time() / 100) * 10  # Dalgalı trend

        return {
            "timestamp": datetime.now().isoformat(),
            "firin_sicaklik": 1400 + int(trend) + random.randint(-20, 20),
            "uretim_hizi": 200 + int(trend/2) + random.randint(-15, 15),
            "enerji_tuketim": 900 + trend + random.uniform(-30, 30),
            "makine_vibrasyon": 5.0 + (trend/20) + random.uniform(-1, 1),
            "hava_nemi": 50 + random.randint(-8, 8),
            "konveyor_hizi": 3.0 + random.uniform(-0.2, 0.2),
            "basinc": 100 + random.randint(-10, 10)
        }

    def gercek_zamanli_kalite_verisi(self):
        """Kalite verisi üret"""
        uretim_hizi = self.sensor_verisi.get('uretim_hizi', 200)
        # Üretim hızı arttıkça kalite düşme eğilimi
        hiz_etkisi = max(0, (uretim_hizi - 200) * 0.01)

        return {
            "timestamp": datetime.now().isoformat(),
            "kusur_orani": round(1.5 + hiz_etkisi + random.uniform(-0.5, 0.5), 2),
            "renk_tutarlilik": round(95 - hiz_etkisi + random.uniform(-2, 2), 1),
            "duvar_kalinligi": round(2.0 + random.uniform(-0.1, 0.1), 2),
            "govde_capi": round(70.0 + random.uniform(-1, 1), 1),
            "yuzey_duzgunluk": round(93 + random.uniform(-3, 3), 1)
        }

    def gercek_zamanli_calisan_verisi(self):
        """Çalışan performans verisi"""
        return {
            "timestamp": datetime.now().isoformat(),
            "calisan_id": random.choice(self.calisanlar),
            "bolum": random.choice(self.bolumler),
            "performans_puani": round(random.uniform(3.5, 4.8), 1),
            "aktif_gorev": random.choice(["Üretim", "Kalite Kontrol", "Bakım", "Temizlik"]),
            "uretim_verimliligi": round(random.uniform(0.85, 1.15), 2),
            "hata_orani": round(random.uniform(0.1, 2.0), 2)
        }

    def dashboard_verisi(self):
        """Tüm dashboard verilerini birleştir"""
        sensor = self.gercek_zamanli_sensor_verisi()
        kalite = self.gercek_zamanli_kalite_verisi()
        calisan = self.gercek_zamanli_calisan_verisi()

        self.sensor_verisi = sensor
        self._veri_gecmisi.append({**sensor, **kalite})

        # Son 50 kaydı tut
        if len(self._veri_gecmisi) > 50:
            self._veri_gecmisi.pop(0)

        return {
            "sensor": sensor,
            "kalite": kalite,
            "calisan": calisan,
            "sistem_durumu": "AKTİF",
            "toplam_uretim": len(self._veri_gecmisi) * 200,
            "ortalama_kalite": np.mean([v['renk_tutarlilik'] for v in self._veri_gecmisi])
        }

# Global instance
colab_fabrika = ColabCamFabrikasi()

In [ ]:
# Flask uygulaması oluştur
app = Flask(__name__)
run_with_ngrok(app)  # Colab için ngrok entegrasyonu

@app.route('/')
def ana_sayfa():
    return """
    <html>
        <head>
            <title>🏭 Camlux Fabrikası - Dijital İkiz</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 40px; }
                .endpoint { background: #f0f0f0; padding: 10px; margin: 10px 0; }
            </style>
        </head>
        <body>
            <h1>🏭 Camlux Fabrikası Dijital İkiz API</h1>
            <p>Colab üzerinde çalışan gerçek zamanlı simülasyon</p>

            <div class="endpoint">
                <h3>📊 API Endpoint'leri:</h3>
                <ul>
                    <li><a href="/api/dashboard">/api/dashboard</a> - Tüm veriler</li>
                    <li><a href="/api/sensor">/api/sensor</a> - Sensör verileri</li>
                    <li><a href="/api/kalite">/api/kalite</a> - Kalite verileri</li>
                    <li><a href="/api/calisan">/api/calisan</a> - Çalışan verileri</li>
                    <li><a href="/api/analiz">/api/analiz</a> - Analiz raporu</li>
                </ul>
            </div>
        </body>
    </html>
    """

@app.route('/api/dashboard')
def dashboard_api():
    return jsonify(colab_fabrika.dashboard_verisi())

@app.route('/api/sensor')
def sensor_api():
    return jsonify(colab_fabrika.gercek_zamanli_sensor_verisi())

@app.route('/api/kalite')
def kalite_api():
    return jsonify(colab_fabrika.gercek_zamanli_kalite_verisi())

@app.route('/api/calisan')
def calisan_api():
    return jsonify(colab_fabrika.gercek_zamanli_calisan_verisi())

@app.route('/api/analiz')
def analiz_api():
    veri = colab_fabrika.dashboard_verisi()

    analiz = {
        "uretim_tahmini": {
            "sonraki_saat": random.randint(190, 210),
            "tahmin_guven": round(random.uniform(0.85, 0.95), 2)
        },
        "kalite_analizi": {
            "mevcut_durum": veri['kalite']['renk_tutarlilik'],
            "hedef": 96.0,
            "durum": "İYİ" if veri['kalite']['renk_tutarlilik'] > 94 else "DÜZELTME GEREKİYOR"
        },
        "enerji_analizi": {
            "tuketim": veri['sensor']['enerji_tuketim'],
            "verimlilik": round(veri['sensor']['uretim_hizi'] / veri['sensor']['enerji_tuketim'], 3),
            "oneri": "Normal" if veri['sensor']['enerji_tuketim'] < 920 else "Enerji Optimizasyonu Gerekli"
        }
    }

    return jsonify(analiz)

print("🚀 Colab Dijital İkiz API'si hazır!")
print("▶️ Aşağıdaki hücreyi çalıştırarak API'yi başlatın...")

In [ ]:
# Gerçek Zamanlı Monitoring ve Dashboard
def gercek_zamanli_monitor(sure_dakika=3):
    """Colab'da gerçek zamanlı monitoring"""
    print(f"🔄 {sure_dakika} dakika boyunca gerçek zamanlı monitoring başlatılıyor...")
    print("   (Her 5 saniyede bir güncellenecek - Durdurmak için: Runtime → Interrupt execution)")

    baslangic = time.time()
    veri_gecmisi = []
    update_count = 0

    try:
        while time.time() - baslangic < sure_dakika * 60:
            # Önceki çıktıyı temizle
            from IPython.display import clear_output
            clear_output(wait=True)

            # API'den veri al
            try:
                response = requests.get('http://localhost:5000/api/dashboard', timeout=5)
                if response.status_code == 200:
                    veri = response.json()
                    veri_gecmisi.append(veri)
                    update_count += 1
                else:
                    print("❌ API'den veri alınamadı, simülasyon verisi kullanılıyor...")
                    veri = colab_fabrika.dashboard_verisi()
                    veri_gecmisi.append(veri)
            except:
                print("❌ API'ye bağlanılamadı, simülasyon verisi kullanılıyor...")
                veri = colab_fabrika.dashboard_verisi()
                veri_gecmisi.append(veri)

            sensor = veri['sensor']
            kalite = veri['kalite']

            # Dashboard göster
            print("="*70)
            print("🏭 CAMLUX FABRİKASI - GERÇEK ZAMANLI DİJİTAL İKİZ")
            print("="*70)

            print(f"\n📊 CANLI VERİLER (Güncelleme #{update_count}):")
            print(f"   🌡️  Fırın Sıcaklığı: {sensor['firin_sicaklik']}°C | Hedef: 1400°C")
            print(f"   ⚙️  Üretim Hızı: {sensor['uretim_hizi']} bardak/saat")
            print(f"   ⚡ Enerji Tüketimi: {sensor['enerji_tuketim']:.1f} kWh")
            print(f"   📈 Makine Titreşimi: {sensor['makine_vibrasyon']:.1f} mm/s")

            print(f"\n🎯 KALİTE METRİKLERİ:")
            print(f"   ✅ Kusur Oranı: %{kalite['kusur_orani']:.1f} | Hedef: <%2.0")
            print(f"   🎨 Renk Tutarlılık: %{kalite['renk_tutarlilik']:.1f}")
            print(f"   📏 Duvar Kalınlığı: {kalite['duvar_kalinligi']:.2f} mm")

            # Verimlilik analizi
            verimlilik = sensor['uretim_hizi'] / sensor['enerji_tuketim']
            print(f"\n📈 VERİMLİLİK ANALİZİ:")
            print(f"   🔄 Enerji Verimliliği: {verimlilik:.3f} bardak/kWh")

            # Trend analizi
            if len(veri_gecmisi) > 1:
                son_sicaklik = veri_gecmisi[-1]['sensor']['firin_sicaklik']
                onceki_sicaklik = veri_gecmisi[-2]['sensor']['firin_sicaklik']
                trend = "📈" if son_sicaklik > onceki_sicaklik else "📉" if son_sicaklik < onceki_sicaklik else "➡️"
                print(f"\n📊 TREND ANALİZİ:")
                print(f"   {trend} Fırın sıcaklığı: {onceki_sicaklik}°C → {son_sicaklik}°C")

            # Alarm durumları
            print(f"\n⚠️  SİSTEM DURUMU:")
            if sensor['firin_sicaklik'] > 1420:
                print("   🔴 KRİTİK: Fırın sıcaklığı çok yüksek!")
            elif sensor['firin_sicaklik'] < 1380:
                print("   🟡 UYARI: Fırın sıcaklığı düşük!")
            else:
                print("   🟢 NORMAL: Fırın sıcaklığı ideal aralıkta")

            if kalite['kusur_orani'] > 2.5:
                print("   🔴 KRİTİK: Kusur oranı çok yüksek!")
            elif kalite['kusur_orani'] > 2.0:
                print("   🟡 UYARI: Kusur oranı yüksek!")
            else:
                print("   🟢 NORMAL: Kusur oranı kabul edilebilir")

            print(f"\n⏰ Son güncelleme: {datetime.now().strftime('%H:%M:%S')}")
            print(f"   Kalan süre: {sure_dakika * 60 - (time.time() - baslangic):.0f} saniye")
            print("   Durdurmak için: Runtime → Interrupt execution")

            # 5 saniye bekle
            time.sleep(5)

    except KeyboardInterrupt:
        print("\n🛑 Monitoring kullanıcı tarafından durduruldu")
    except Exception as e:
        print(f"\n❌ Monitoring hatası: {e}")

    # Sonuç özeti
    print(f"\n📋 MONITORING ÖZETİ:")
    print(f"   Toplam güncelleme: {update_count}")
    print(f"   Toplam süre: {time.time() - baslangic:.1f} saniye")
    if veri_gecmisi:
        ortalama_uretim = np.mean([v['sensor']['uretim_hizi'] for v in veri_gecmisi])
        ortalama_kalite = np.mean([v['kalite']['renk_tutarlilik'] for v in veri_gecmisi])
        print(f"   Ortalama Üretim: {ortalama_uretim:.1f} bardak/saat")
        print(f"   Ortalama Kalite: %{ortalama_kalite:.1f}")

# Monitoring'i başlat
print("🎯 Gerçek zamanlı monitoring başlatılıyor...")
gercek_zamanli_monitor(sure_dakika=3)

In [ ]:
import plotly.io as pio
pio.renderers.default = 'colab'  # Colab için plotly ayarı

def colab_dashboard_olustur():
    """Colab'da çalışan basit dashboard"""

    # API'den veri çek
    try:
        # Localhost yerine public ngrok URL'si kullan
        time.sleep(2)
        response = requests.get('http://localhost:5000/api/dashboard')
        data = response.json()
    except:
        print("❌ API'ye bağlanılamadı. Simülasyon verisi kullanılıyor...")
        data = colab_fabrika.dashboard_verisi()

    sensor = data['sensor']
    kalite = data['kalite']

    print("="*70)
    print("🏭 CAMLUX FABRİKASI - COLAB DİJİTAL İKİZ DASHBOARD")
    print("="*70)

    # Metrikler
    print(f"\n📊 GERÇEK ZAMANLI METRİKLER:")
    print(f"   🌡️  Fırın Sıcaklığı: {sensor['firin_sicaklik']}°C | Hedef: 1400°C")
    print(f"   ⚙️  Üretim Hızı: {sensor['uretim_hizi']} bardak/saat")
    print(f"   ⚡ Enerji Tüketimi: {sensor['enerji_tuketim']:.1f} kWh")
    print(f"   📈 Makine Titreşimi: {sensor['makine_vibrasyon']:.1f} mm/s")

    print(f"\n🎯 KALİTE METRİKLERİ:")
    print(f"   ✅ Kusur Oranı: %{kalite['kusur_orani']:.1f} | Hedef: <%2.0")
    print(f"   🎨 Renk Tutarlılık: %{kalite['renk_tutarlilik']:.1f}")
    print(f"   📏 Duvar Kalınlığı: {kalite['duvar_kalinligi']:.1f} mm")

    print(f"\n📈 VERİMLİLİK ANALİZİ:")
    verimlilik = sensor['uretim_hizi'] / sensor['enerji_tuketim']
    print(f"   🔄 Enerji Verimliliği: {verimlilik:.2f} bardak/kWh")
    print(f"   📦 Toplam Üretim: {data['toplam_uretim']} bardak")
    print(f"   🏆 Ortalama Kalite: %{data['ortalama_kalite']:.1f}")

    # Durum göstergeleri
    print(f"\n⚠️  SİSTEM DURUMU:")
    durum_sicaklik = "✅ NORMAL" if 1380 <= sensor['firin_sicaklik'] <= 1420 else "⚠️  DİKKAT"
    durum_kusur = "✅ İYİ" if kalite['kusur_orani'] < 2.0 else "🔴 KRİTİK"
    durum_titresim = "✅ NORMAL" if sensor['makine_vibrasyon'] < 7.0 else "⚠️  UYARI"

    print(f"   Fırın Sıcaklığı: {durum_sicaklik}")
    print(f"   Kusur Oranı: {durum_kusur}")
    print(f"   Makine Titreşimi: {durum_titresim}")

    return data

# İlk dashboard'u göster
veri = colab_dashboard_olustur()

In [ ]:
def gercek_zamanli_monitor(sure_dakika=5):
    """Belirli süre boyunca gerçek zamanlı monitoring"""
    print(f"\n🔄 {sure_dakika} dakika boyunca gerçek zamanlı monitoring başlatılıyor...")
    print("   (Her 10 saniyede bir güncellenecek)")

    baslangic = time.time()
    veri_gecmisi = []

    while time.time() - baslangic < sure_dakika * 60:
        try:
            # Önceki çıktıyı temizle (Colab için)
            from IPython.display import clear_output
            clear_output(wait=True)

            # Yeni veri al ve dashboard göster
            veri = colab_dashboard_olustur()
            veri_gecmisi.append(veri)

            # Trend analizi
            if len(veri_gecmisi) > 1:
                son_sicaklik = veri_gecmisi[-1]['sensor']['firin_sicaklik']
                onceki_sicaklik = veri_gecmisi[-2]['sensor']['firin_sicaklik']
                trend = "📈 ARTIYOR" if son_sicaklik > onceki_sicaklik else "📉 AZALIYOR" if son_sicaklik < onceki_sicaklik else "➡️ SABİT"
                print(f"\n📊 TREND: Fırın sıcaklığı {trend}")

            print(f"\n⏰ Son güncelleme: {datetime.now().strftime('%H:%M:%S')}")
            print("   (Durdurmak için: Ctrl + M .)")

            # 10 saniye bekle
            time.sleep(10)

        except KeyboardInterrupt:
            print("\n🛑 Monitoring durduruldu")
            break
        except Exception as e:
            print(f"\n❌ Hata: {e}")
            time.sleep(5)

# Monitoring'i başlat
print("🎯 Gerçek zamanlı monitoring için aşağıdaki hücreyi çalıştırın:")

In [ ]:
# 3 dakikalık monitoring başlat
gercek_zamanli_monitor(sure_dakika=3)

In [ ]:
# Geçmiş verileri analiz et
def veri_analizi_raporu():
    """Toplanan verilerin analiz raporu"""
    print("\n" + "="*60)
    print("📊 VERİ ANALİZ RAPORU")
    print("="*60)

    # Simüle edilmiş geçmiş veri
    gunluk_veri = []
    for i in range(24):  # 24 saatlik veri
        gunluk_veri.append(colab_fabrika.dashboard_verisi())
        time.sleep(0.1)  # Küçük gecikme

    df = pd.DataFrame([v['sensor'] for v in gunluk_veri])
    df_kalite = pd.DataFrame([v['kalite'] for v in gunluk_veri])

    print(f"\n📈 İSTATİSTİKLER:")
    print(f"   • Ortalama Üretim Hızı: {df['uretim_hizi'].mean():.1f} bardak/saat")
    print(f"   • Maksimum Sıcaklık: {df['firin_sicaklik'].max()}°C")
    print(f"   • Minimum Sıcaklık: {df['firin_sicaklik'].min()}°C")
    print(f"   • Ortalama Kusur Oranı: %{df_kalite['kusur_orani'].mean():.2f}")
    print(f"   • Enerji Verimliliği: {(df['uretim_hizi'] / df['enerji_tuketim']).mean():.3f}")

    # Basit grafik
    fig = make_subplots(rows=2, cols=2, subplot_titles=('Fırın Sıcaklığı', 'Üretim Hızı', 'Enerji Tüketimi', 'Kusur Oranı'))

    fig.add_trace(go.Scatter(y=df['firin_sicaklik'], name='Sıcaklık'), row=1, col=1)
    fig.add_trace(go.Scatter(y=df['uretim_hizi'], name='Üretim'), row=1, col=2)
    fig.add_trace(go.Scatter(y=df['enerji_tuketim'], name='Enerji'), row=2, col=1)
    fig.add_trace(go.Scatter(y=df_kalite['kusur_orani'], name='Kusur'), row=2, col=2)

    fig.update_layout(height=600, showlegend=False, title_text="24 Saatlik Üretim Trendleri")
    fig.show()

# Analiz raporunu göster
veri_analizi_raporu()